## Project prep
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
    - cohort pulled from the sql table 
    - sort all the endpoints 
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
    - divided web dev and data science (4 different programs)
    - sort all the endpoints 
    - counts of cohorts accessing same endpoints
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
    - active status indicator 
    - filter by the student being active 
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
    - figure out what constitutes a suspicious IP address
        - any ip addresses from china?
        - figure out general area of ip address
    - how to recognize web scraping
    - SKIP 
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
    - how to classify the endpoints as web dev and not this might be crazy
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
    - active status indicator 
    - each row of a ping would have student_active column - Parker 
    - 
7. Which lessons are least accessed?
    - kind of related to 2
8. Anything else I should be aware of?

1 is related to 7 and 2 

3 is relate to 6

anything extra we find

### Data Frame Must Haves
- log data
- cohort data
- Join on cohort id (Left Join) - Lupe has the join and the drop columns 
- column that has the active student at time of request (yes or no)
- need key for all the program ids (is_data_science maybe) 
- endpoint as a web dev, or data science 
    - Anything came before ada's start date is webdev


In [55]:
# prepare file for anomaly detection exercises
import pandas as pd
import numpy as np

# Vis tools
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans 
from datetime import datetime
# defining some functions to make it easier. will go in Wrangle function
from env import host, password, username
import os
import prepare

In [2]:
###################### Getting database Url ################
def get_db_url(db_name, user=username, host=host, password=password):
    """
        This helper function takes as default the user host and password from the env file.
        You must input the database name. It returns the appropriate URL to use in connecting to a database.
    """
    url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'
    return url

######################### get generic data #########################
def get_any_data(database, sql_query):
    '''
    put in the query and the database and get the data you need in a dataframe
    '''

    return pd.read_sql(sql_query, get_db_url(database))

In [3]:
def acquire_curriculum_logs(csv_name = "anonymized-curriculum-access-07-2021.txt"):
    '''
    This function reads the csv of curriculum access logs
    If the file name needs to be changed change the default arguement
    '''
    # assign column names to use
    colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
    # read csv
    df = pd.read_csv(csv_name, 
                 sep="\s", 
                 header=None, 
                 names = colnames, 
                 usecols=[0, 2, 3, 4, 5])
    return df

def make_datetime_index(df, col_name):
    '''
    This function takes in a dataframe 
    A column name of the column that is your date (as string)
    Performs basic to_datetime conversion and sets tha column as the index
    '''
    
    df[col_name] = pd.to_datetime(df[col_name])

    df = df.set_index(col_name)
    
    return df

def prep(df):
    '''
    This function takes in the curriculum log data, and the user number you want to look at
    returns a dataframe of the pages they accessed and when
    '''
    #df = df[df.user_id == user]
    df = make_datetime_index(df, 'date')
    pages = df['endpoint'].resample('d').count()
    return pages

In [4]:
df = pd.read_csv('anonymized-curriculum-access.csv')

In [5]:
dbase = 'curriculum_logs'
sqlquery = 'SELECT * FROM cohorts'

cohorts = get_any_data(dbase, sqlquery)

In [6]:
log_df = df.merge(cohorts, how = 'left', left_on = 'cohort_id', right_on = 'id')

In [7]:
log_df.sample(50)

,Unnamed: 0,date,endpoint,user_id,cohort_id,source_ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
262563,262563,2019-05-09,spring/fundamentals/security,64,28.0,97.105.19.58,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
175356,175356,2018-12-24,java-i,302,27.0,192.171.117.210,27.0,Yosemite,#yosemite,2018-11-05,2019-04-03,2018-11-05 15:26:37,2018-11-05 15:26:37,None,2.0
807302,807302,2021-02-02,sql/mysql-overview,692,59.0,96.8.130.244,59.0,Darden,#darden,2020-07-13,2021-01-12,2020-07-13 18:32:19,2020-07-13 18:32:19,None,3.0
101758,101758,2018-08-09,mysql/users,198,24.0,97.105.19.58,24.0,Voyageurs,#voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,None,2.0
905168,905168,2021-04-26,anomaly-detection/detecting-with-clustering,835,133.0,173.173.109.5,133.0,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,None,3.0
249878,249878,2019-04-22,java-iii/servlets,341,29.0,97.105.19.58,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,None,2.0
199678,199678,2019-02-15,2.01.00_Descriptive_Stats,363,NaN,97.105.19.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557992,557992,2020-05-29,html-css/css-i/selectors-and-properties,667,58.0,68.203.194.78,58.0,Hyperion,#hyperion,2020-05-26,2020-11-10,2020-05-26 19:22:44,2020-05-26 19:22:44,None,2.0
460399,460399,2020-02-19,javascript-i,564,53.0,97.105.19.58,53.0,Fortuna,#fortuna,2020-01-13,2020-06-23,2020-01-13 21:17:08,2020-01-13 21:17:08,None,2.0
818463,818463,2021-02-11,java-ii/file-io,793,132.0,99.137.94.0,132.0,Kalypso,#kalypso,2020-11-02,2021-05-04,2020-11-02 20:43:58,2020-11-02 20:43:58,None,2.0


In [8]:
log_df = log_df.drop(columns = ['Unnamed: 0','slack','created_at','updated_at','deleted_at'])

In [9]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1018810 entries, 0 to 1018809
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   date        1018810 non-null  object 
 1   endpoint    1018809 non-null  object 
 2   user_id     1018810 non-null  int64  
 3   cohort_id   965313 non-null   float64
 4   source_ip   1018810 non-null  object 
 5   id          954313 non-null   float64
 6   name        954313 non-null   object 
 7   start_date  954313 non-null   object 
 8   end_date    954313 non-null   object 
 9   program_id  954313 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 85.5+ MB


In [10]:
log_df = make_datetime_index(log_df,'date' )

In [11]:
log_df.isnull().value_counts()

endpoint  user_id  cohort_id  source_ip  id     name   start_date  end_date  program_id
False     False    False      False      False  False  False       False     False         954312
                   True       False      True   True   True        True      True           53497
                   False      False      True   True   True        True      True           11000
True      False    False      False      False  False  False       False     False              1
dtype: int64

In [12]:
log_df = log_df.dropna()

In [13]:
log_df.shape

(954312, 9)

In [19]:
log_df.start_date = pd.to_datetime(log_df.start_date.copy())

In [18]:
log_df.end_date = pd.to_datetime(log_df.end_date.copy())

In [29]:
log_df['is_active'] = (log_df.index >= log_df.start_date) & (log_df.index <= log_df.end_date)

In [31]:
log_df.sample(50)

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id,is_active
date,,,,,,,,,,
2020-08-25,java-i,189,24.0,67.11.82.179,24.0,Voyageurs,2018-05-29,2018-10-11,2.0,False
2021-04-06,fundamentals/intro-to-data-science,903,137.0,70.117.8.141,137.0,Florence,2021-03-15,2021-09-03,3.0,True
2020-07-24,/,670,58.0,65.43.116.84,58.0,Hyperion,2020-05-26,2020-11-10,2.0,True
2020-02-06,search/search_index.json,552,53.0,97.105.19.58,53.0,Fortuna,2020-01-13,2020-06-23,2.0,True
2021-06-06,jquery/events/mouse-events,975,139.0,18.222.134.219,139.0,Oberon,2021-04-12,2021-10-01,2.0,True
2018-11-14,java-ii/inheritance-and-polymorphism,277,26.0,97.105.19.58,26.0,Xanadu,2018-09-17,2019-02-08,2.0,True
2021-04-12,classification/classical_programming_vs_machin...,581,28.0,70.112.179.142,28.0,Staff,2014-02-04,2014-02-04,2.0,False
2020-11-09,javascript-i/conditionals,805,132.0,72.179.174.165,132.0,Kalypso,2020-11-02,2021-05-04,2.0,True
2021-07-09,jquery/personal-site,995,1.0,104.51.42.150,1.0,Arches,2014-02-04,2014-04-22,1.0,False


In [32]:
active_df = log_df[log_df.is_active == True]

In [37]:
active_df.groupby(by = 'user_id').sum()

,cohort_id,id,program_id,is_active
user_id,,,,
2,33902.0,33902.0,3082.0,1541
3,33308.0,33308.0,3028.0,1514
4,15224.0,15224.0,1384.0,692
5,35662.0,35662.0,3242.0,1621
6,28886.0,28886.0,2626.0,1313
...,...,...,...,...
991,122181.0,122181.0,1758.0,879
993,47260.0,47260.0,680.0,340
994,26966.0,26966.0,388.0,194


In [38]:
inactive_df = log_df[log_df.is_active == False]

In [51]:
(inactive_df[inactive_df['program_id'] == 3]).endpoint.value_counts().head(30)

/                                                                1782
search/search_index.json                                          649
classification/overview                                           368
sql/mysql-overview                                                343
classification/scale_features_or_not.svg                          325
anomaly-detection/AnomalyDetectionCartoon.jpeg                    226
anomaly-detection/overview                                        225
fundamentals/AI-ML-DL-timeline.jpg                                223
fundamentals/modern-data-scientist.jpg                            221
fundamentals/intro-to-data-science                                217
classification/classical_programming_vs_machine_learning.jpeg     187
1-fundamentals/modern-data-scientist.jpg                          130
1-fundamentals/AI-ML-DL-timeline.jpg                              129
1-fundamentals/1.1-intro-to-data-science                          129
classification/prep 

In [52]:
def active_status(df, start, end):
    df['is_active'] = (df.index >= df[start]) & (df.index <= df[end])
    return df

In [54]:
active_status(log_df, 'start_date', 'end_date')

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id,is_active
date,,,,,,,,,,
2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0,True
...,...,...,...,...,...,...,...,...,...,...
2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0,True


In [56]:
prepare.active_status(log_df, log_df.index, 'start_date','end_date')

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id,is_active
date,,,,,,,,,,
2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0,True
...,...,...,...,...,...,...,...,...,...,...
2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
